# Optimization bonus - fine tuning for NSG

[Optimize NSG](#1-optimize-nsg)

# Import libraries

In [ ]:
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

import pandas

from params import nsg_test

In [1]:
input_path = b'../MNIST/input.dat'
query_path = b'../MNIST/query.dat'

# Optimize NSG

To skip logs, click [here](#visualize-nsg-study-results).

In [ ]:
def objective_nsg(trial):
    param_dict = {'m' : trial.suggest_int('m', 3, 500),
                  'l' : trial.suggest_int('l', 10, 1000),
                  'lq': trial.suggest_int('lq', 1, 1000),
                  'k' : trial.suggest_int('k', 40, 100)}
    
    print("Trial parameters:", param_dict)

    average_time, maf = nsg_test(input_path, query_path, queries_num=100, **param_dict, N=5)

    return maf.value, average_time.value

In [ ]:
%%time
for i in range(10):
    try:
        nsg_study = optuna.create_study(study_name='nsg', directions=['minimize', 'minimize'])
        nsg_study.optimize(objective_nsg, n_trials=50, n_jobs=-1)
        print("-------------------- Best trials --------------------")
        trials = sorted(nsg_study.best_trials, key=lambda x: x.values)
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}".format(trial.values))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

### Visualize NSG study results

In [ ]:
plot_pareto_front(nsg_study, target_names=['maf', 'average_time'])

In [ ]:
plot_optimization_history(nsg_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_optimization_history(nsg_study, target = lambda t: t.values[1], target_name = 'average_time')

In [ ]:
plot_slice(nsg_study, target = lambda t: t.values[0], target_name = 'maf')

In [ ]:
plot_slice(nsg_study, target = lambda t: t.values[1], target_name = 'average_time')